In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score
import random

In [14]:
def load_feature_data(feature_name):
    BATCH_SIZE = 32
    IMG_SIZE = (224, 224)
    feature = tf.keras.utils.image_dataset_from_directory("feature_images_30x30/{0}/test".format(feature_name),
                                                                batch_size=BATCH_SIZE,
                                                                image_size=IMG_SIZE,
                                                                shuffle=False)
    return feature

def load_network(network_name,feature_model):
    if (network_name=="NasNet"):
        model = tf.keras.models.load_model('{0}/{1}.h5'.format(network_name,feature_model), compile=False)
    else:
        model = tf.keras.models.load_model('{0}/{1}.h5'.format(network_name,feature_model))
    return model

def evaluation_prob(dataset,model):
    val_images =[]
    true_labels = []
    for image, label in dataset.unbatch():
        val_images.append(image)
        true_labels.append(label)
    val_images = np.array(val_images)
    true_labels_arr = np.array(true_labels)
    predictions_prob = model.predict(val_images)
    return predictions_prob,true_labels_arr

def calculate_acc(predictions,true_labels):
    predict_labels = predictions.argmax(axis=1)
    return accuracy_score(true_labels,predict_labels)


In [15]:
features = ['D','S','A','H','DS','DA','DH','SA','SH','AH','DSA','DSH','DAH','SAH']
networks = ['MobileNet-','EfficientNet-','ResNet-','NasNet-','DenseNet-']
datasets = {}
MobileNet_models = {}
EfficientNet_models = {}
ResNet_models = {}
NasNet_models = {}
DenseNet_models = {}
margin_accuracy_vector = {}

In [16]:
def load_feature_data_network(feature):
    datasets[feature] = load_feature_data(feature)
    MobileNet_models[feature] = load_network("MobileNet",feature)
    EfficientNet_models[feature] = load_network("EfficientNet",feature)
    ResNet_models[feature] = load_network("ResNet",feature)
    NasNet_models[feature] = load_network("NasNet",feature)
    DenseNet_models[feature] = load_network("DenseNet",feature)

In [7]:
# load all networks and data
for i in features:
    load_feature_data_network(i)
# initalize all models vector
for i in networks:
    for j in features:
        # compute the original accuracy of every model
        if(i == 'MobileNet-'):
            prob,true_label = evaluation_prob(datasets[j],MobileNet_models[j])
        elif(i=='EfficientNet-'):
            prob,true_label = evaluation_prob(datasets[j],EfficientNet_models[j])
        elif(i == 'ResNet-'):
            prob,true_label = evaluation_prob(datasets[j],ResNet_models[j])
        elif(i=='NasNet-'):
            prob,true_label = evaluation_prob(datasets[j],NasNet_models[j])
        elif(i=='DenseNet-'):
            prob,true_label = evaluation_prob(datasets[j],DenseNet_models[j])            
        
        accuracy = calculate_acc(prob,true_label)
        margin_accuracy_vector[i+j] = [accuracy,1]           # [accuracy , count] 

In [25]:
# the number of time run the random number of models
num_of_trail = 100
# all models names
models_names = margin_accuracy_vector.keys()
max_accur = 0 
for i in range(0,num_of_trail):
    num_of_models =random.randint(2,10)
    # get random model
    random_models= random.sample(list(models_names),num_of_models)
    # accuracy of all models
    probability_of_all = 0
    for i in random_models:
        model_feature = i.split('-')        # split the name to the network and feature
        f = model_feature[1]
        if(model_feature[0] == 'MobileNet'):
            prob,true_label = evaluation_prob(datasets[f],MobileNet_models[f])
        elif(model_feature[0]=='EfficientNet'):
            prob,true_label = evaluation_prob(datasets[f],EfficientNet_models[f])
        elif(model_feature[0] == 'ResNet'):
            prob,true_label = evaluation_prob(datasets[f],ResNet_models[f])
        elif(model_feature[0]=='NasNet'):
            prob,true_label = evaluation_prob(datasets[f],NasNet_models[f])
        elif(model_feature[0]=='DenseNet'):
            prob,true_label = evaluation_prob(datasets[f],DenseNet_models[f])
        probability_of_all += np.log(prob)
    accuracy = calculate_acc(probability_of_all,true_label)
    if(accuracy>max_accur):
        max_combination = random_models
        max_accur = accuracy      
    # add accuracy to the randome models and increase counter
    for i in random_models:
        margin_accuracy_vector[i][0] += accuracy
        margin_accuracy_vector[i][1] += 1

In [ ]:
models_margin_accuracy = {}
models_names = margin_accuracy_vector.keys() 
for i in models_names:
    models_margin_accuracy[i] = margin_accuracy_vector[i][0] / margin_accuracy_vector[i][1]

In [ ]:
combinations_dict = {}
# sort models by margin accuracy value
ranked_models = sorted(models_margin_accuracy.items(), key=lambda x:x[1],reverse=True)
# perform step wise
for j in range(1,71):
    step_wise_models = []
    for i in range(0,j):        
        step_wise_models.append(ranked_models[i][0])
    probability_of_all = 0
    for model in step_wise_models:
        model_feature = model.split('-')        # split the name to the network and feature
        f = model_feature[1]
        if(model_feature[0] == 'MobileNet'):
            prob,true_label = evaluation_prob(datasets[f],MobileNet_models[f])
        elif(model_feature[0]=='EfficientNet'):
            prob,true_label = evaluation_prob(datasets[f],EfficientNet_models[f])
        elif(model_feature[0] == 'ResNet'):
            prob,true_label = evaluation_prob(datasets[f],ResNet_models[f])
        elif(model_feature[0]=='NasNet'):
            prob,true_label = evaluation_prob(datasets[f],NasNet_models[f])
        elif(model_feature[0]=='DenseNet'):
            prob,true_label = evaluation_prob(datasets[f],DenseNet_models[f])
        probability_of_all += np.log(prob)
    accuracy = calculate_acc(probability_of_all,true_label)
    combinations_dict[str(j)] = [step_wise_models,accuracy] 